In [ ]:
from file_converter import JPG, PNG


jpg = JPG('media/input_9.jpg')
png_bytes = jpg.convert_to_png()
with open('media/output/text.docx', 'wb') as outfile:
    outfile.write(png_bytes.getbuffer())

png = PNG(png_bytes)
png.img.show()

# example of converting image and show it

In [2]:
from file_converter import PDF


docx = PDF('media/input_4.pdf')
pdf = docx.convert_to_docx()
with open('media/output/text.docx', 'wb') as outfile:
    outfile.write(pdf.getbuffer())

# example of Error while converting file (cannot convert pdf to docx for now)
# Why? idk

BaseException: Error while converting file

In [ ]:
from file_converter import DOCX


docx = DOCX('media/input.docx')
pdf = docx.convert_to_pdf()
with open('media/output/test.pdf', 'wb') as outfile:
    outfile.write(pdf.getbuffer())

In [ ]:
formats = {
    'docx': {
        'pdf': {},
        'odt': {},
        'html': {},
        'txt': {},
        'rtf': {}
    },
    'pdf': {
        'docx': {},
        'xlsx': {},
        'pptx': {},
        'odt': {},
        'ods': {},
        'html': {},
        'txt': {}
    },
    'png': {
        'jpg': {},
        'bmp': {},
        'tiff': {},
    },
    'jpg': {
        'png': {},
        'bmp': {},
        'tiff': {},
    }
}

In [ ]:
import os
from file_converter import define_class_for_format


dir_path = 'media'
listdir = os.listdir(dir_path)
listdir.sort()
for path in listdir:
    if not os.path.isfile(os.path.join(dir_path, path)):
        continue
    
    input_name, input_format = path.split('.')
    if input_format not in formats:
        continue

    format_class = define_class_for_format(input_format)
    dest_formats = formats[input_format]
    input_obj = format_class(f'{dir_path}/{path}')

    success_formats = []
    for dest_format, data in dest_formats.items():
        try:
            output_obj = input_obj.convert_to(dest_format)
        except:
            ...
        else:
            with open(f'media/output/output_{input_name.split("_")[1]}.{dest_format}', 'wb') as outfile:
                outfile.write(output_obj.getbuffer())
            success_formats.append(dest_format)

    print(f"{path} -> {', '.join(success_formats)}")
    